# GRU Keyboard Model (2026 Optimized)

Train a GRU model with **2026 best practices** for keyboard suggestions.

**Supports 3 Tasks (Hybrid System):**

| Input Format | Method | Example |
|--------------|--------|----------|
| `text + space` | GRU Model | "How are " → you, they, we |
| `partial word` | Vocab + GRU Rerank | "Hel" → Hello, Help |
| `typo word` | Edit Dist + GRU Rerank | "Thers" → There |

**2026 Upgrades:**
- ✅ GRU Reranking - Context-aware typo/completion
- ✅ Perplexity Metric - Better quality measurement
- ✅ INT8 Quantization - 75% smaller model
- ✅ AdamW Optimizer - Better regularization
- ✅ Mixed Precision - 2x faster training

**Expected Results:**
- Accuracy: 85-90%
- Model Size: <10MB (INT8)
- Inference: <5ms on mobile

---

**Instructions:**
1. Runtime → Change runtime type → GPU (T4)
2. Set `TESTING_MODE = True` for quick test
3. Set `TESTING_MODE = False` for full training
4. Run all cells in order
5. Download TFLite for mobile

## 1. Environment Setup

In [ ]:
# Mount Google Drive and setup directories
from google.colab import drive
import os

drive.mount('/content/drive')

# Define directories
DRIVE_DIR = '/content/drive/MyDrive/Keyboard-Suggestions-ML-Colab'
os.makedirs(f"{DRIVE_DIR}/models/gru_keyboard", exist_ok=True)

print(f"✓ Google Drive mounted")
print(f"✓ Project directory: {DRIVE_DIR}")

In [ ]:
# Install dependencies
!pip install -q tensorflow keras nltk pandas numpy scikit-learn tqdm
print("✓ Dependencies installed")

In [ ]:
# ============================================================
# CONFIGURATION - OPTIMIZED FOR T4 GPU
# ============================================================

TESTING_MODE = True  # ← Change to False for full training

if TESTING_MODE:
    print("⚠️  TESTING MODE")
    print("   - Dataset: keyboard_training_data.txt")
    print("   - Epochs: 2 (quick verification)")
    print("   - Time: ~1 min")
    NUM_EPOCHS = 2
    BATCH_SIZE = 512  # Optimized for T4 GPU
    VOCAB_SIZE_LIMIT = 25000  # Limit vocab for speed
    SEQUENCE_LENGTH = 10  # Better context
else:
    print("✓ FULL TRAINING MODE")
    print("   - Dataset: Fake.csv + True.csv + 1661-0.txt")
    print("   - Epochs: 20")
    print("   - Time: ~8-10 min (with optimizations)")
    NUM_EPOCHS = 20
    BATCH_SIZE = 512  # Optimized for T4 GPU
    VOCAB_SIZE_LIMIT = 25000  # Smaller model, faster inference
    SEQUENCE_LENGTH = 10  # Better predictions

print(f"\nOptimizations:")
print(f"  - Batch size: {BATCH_SIZE} (maximizes GPU)")
print(f"  - Vocab limit: {VOCAB_SIZE_LIMIT:,} (reduces model size)")
print(f"  - Sequence length: {SEQUENCE_LENGTH} (better context)")
print("="*60)

## 2. Verify Datasets in Google Drive

**Required datasets in Google Drive:**
- `{DRIVE_DIR}/datasets/Fake.csv` - Fake news dataset
- `{DRIVE_DIR}/datasets/True.csv` - True news dataset
- `{DRIVE_DIR}/datasets/1661-0.txt` - Next-word prediction corpus

Upload these files to your Google Drive before running.

In [ ]:
import os

print("Checking datasets in Google Drive...")
print("="*60)

# Define dataset paths
FAKE_NEWS_PATH = f"{DRIVE_DIR}/datasets/Fake.csv"
TRUE_NEWS_PATH = f"{DRIVE_DIR}/datasets/True.csv"
CORPUS_PATH = f"{DRIVE_DIR}/datasets/1661-0.txt"

# Check each dataset
datasets_ok = True

for name, path in [("Fake.csv", FAKE_NEWS_PATH), 
                    ("True.csv", TRUE_NEWS_PATH),
                    ("1661-0.txt", CORPUS_PATH)]:
    if os.path.exists(path):
        size = os.path.getsize(path) / (1024 * 1024)
        print(f"✓ {name}: {size:.2f}MB")
    else:
        print(f"✗ Missing: {name}")
        print(f"   Expected at: {path}")
        datasets_ok = False

if not datasets_ok:
    print("\n⚠️  Please upload missing datasets to Google Drive!")
    print(f"   Upload to: {DRIVE_DIR}/datasets/")
    raise FileNotFoundError("Required datasets not found in Google Drive")
else:
    print("\n✅ All datasets found!")

In [ ]:
import pandas as pd
import numpy as np

print("Loading datasets from Google Drive...")
print("="*60)

all_text = []

if TESTING_MODE:
    # Testing mode: Use keyboard_training_data.txt (smaller, faster)
    print("⚠️  TESTING MODE: Using keyboard_training_data.txt")
    
    CORPUS_PATH = f"{DRIVE_DIR}/datasets/keyboard_training_data.txt"
    
    if not os.path.exists(CORPUS_PATH):
        print(f"\n✗ Missing: keyboard_training_data.txt")
        print(f"   Expected at: {CORPUS_PATH}")
        raise FileNotFoundError("keyboard_training_data.txt not found")
    
    with open(CORPUS_PATH, 'r', encoding='utf-8') as f:
        corpus_text = f.read()
    
    all_text.append(corpus_text)
    print(f"✓ Loaded: {len(corpus_text):,} characters")
    
else:
    # Full training mode: Use Fake.csv + True.csv + 1661-0.txt
    print("✓ FULL TRAINING: Using Fake.csv + True.csv + 1661-0.txt")
    
    FAKE_NEWS_PATH = f"{DRIVE_DIR}/datasets/Fake.csv"
    TRUE_NEWS_PATH = f"{DRIVE_DIR}/datasets/True.csv"
    CORPUS_PATH = f"{DRIVE_DIR}/datasets/1661-0.txt"
    
    # Check files exist
    for name, path in [("Fake.csv", FAKE_NEWS_PATH), 
                        ("True.csv", TRUE_NEWS_PATH),
                        ("1661-0.txt", CORPUS_PATH)]:
        if not os.path.exists(path):
            print(f"\n✗ Missing: {name}")
            print(f"   Expected at: {path}")
            raise FileNotFoundError(f"{name} not found")
    
    # Load fake news
    fake_df = pd.read_csv(FAKE_NEWS_PATH)
    true_df = pd.read_csv(TRUE_NEWS_PATH)
    
    print(f"✓ Loaded {len(fake_df):,} fake news articles")
    print(f"✓ Loaded {len(true_df):,} true news articles")
    
    all_text.extend(fake_df['text'].tolist())
    all_text.extend(true_df['text'].tolist())
    
    # Load corpus
    with open(CORPUS_PATH, 'r', encoding='utf-8') as f:
        corpus_text = f.read()
        all_text.append(corpus_text)
    
    print(f"✓ Loaded corpus: {len(corpus_text):,} characters")

# Combine and clean
combined_text = ' '.join(all_text).lower()
combined_text = combined_text.replace('\n', ' ')
combined_text = ' '.join(combined_text.split())

print(f"\n✓ Total: {len(combined_text):,} characters")
print(f"✓ Sample: {combined_text[:200]}...")
print("="*60)

## 4. Tokenize and Create Sequences

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np

print("Tokenizing with vocabulary limit...")
print("="*60)

# Tokenize with vocab limit
tokenizer = Tokenizer(num_words=VOCAB_SIZE_LIMIT)
tokenizer.fit_on_texts([combined_text])

vocab_size = min(len(tokenizer.word_index) + 1, VOCAB_SIZE_LIMIT)

print(f"✓ Total unique words: {len(tokenizer.word_index):,}")
print(f"✓ Vocabulary size (limited): {vocab_size:,}")

# Convert to sequences
sequences = tokenizer.texts_to_sequences([combined_text])[0]

print(f"\nCreating optimized tf.data pipeline...")

sequences_array = np.array(sequences)

# Create dataset using timeseries for next-word prediction
dataset = tf.keras.utils.timeseries_dataset_from_array(
    data=sequences_array[:-1],
    targets=sequences_array[SEQUENCE_LENGTH:],
    sequence_length=SEQUENCE_LENGTH,
    sequence_stride=1,
    shuffle=True,
    batch_size=BATCH_SIZE,
    seed=42
)

# Calculate total steps
total_sequences = len(sequences) - SEQUENCE_LENGTH
total_steps = total_sequences // BATCH_SIZE

# Split: 90% train, 10% validation
val_steps = max(1, total_steps // 10)
train_steps = total_steps - val_steps

# Split dataset with proper steps
train_dataset = dataset.take(train_steps).prefetch(tf.data.AUTOTUNE)
val_dataset = dataset.skip(train_steps).take(val_steps).prefetch(tf.data.AUTOTUNE)

print(f"✓ Total sequences: {total_sequences:,}")
print(f"✓ Total steps: {total_steps:,}")
print(f"✓ Train steps: {train_steps:,} (90%)")
print(f"✓ Val steps: {val_steps:,} (10%)")
print(f"✓ Batch size: {BATCH_SIZE}")
print(f"✓ Prefetching: Enabled")
print("="*60)

print("\n📝 Note: GRU model trains on NEXT-WORD prediction only.")
print("   Word completion & typo correction use vocabulary + edit distance.")

## 5. Build GRU Model

In [ ]:
from tensorflow.keras import mixed_precision
import tensorflow as tf

# Enable Mixed Precision for T4 GPU
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Custom Perplexity Metric (2026 Best Practice)
class Perplexity(tf.keras.metrics.Metric):
    """Measures how 'surprised' the model is. Lower = better."""
    def __init__(self, name='perplexity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=False, reduction='none'
        )
        self.total_loss = self.add_weight(name='total_loss', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        loss = self.cross_entropy(y_true, y_pred)
        self.total_loss.assign_add(tf.reduce_sum(loss))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    
    def result(self):
        return tf.exp(self.total_loss / self.count)
    
    def reset_state(self):
        self.total_loss.assign(0.)
        self.count.assign(0.)

print("="*60)
print("PERFORMANCE OPTIMIZATIONS (2026)")
print("="*60)
print("✓ Mixed Precision enabled (FP16) - 2x faster")
print("✓ Perplexity metric added - Better quality measurement")
print("="*60)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Dropout
from tensorflow.keras.optimizers import AdamW

print("Building GRU model (Functional API + Mixed Precision)...")
print("="*60)

# Input layer
inputs = Input(shape=(SEQUENCE_LENGTH,), name='input')

# Embedding layer
x = Embedding(
    input_dim=vocab_size,
    output_dim=128,
    name='embedding'
)(inputs)

# GRU layer
x = GRU(
    units=256,
    dropout=0.2,
    recurrent_dropout=0.2,
    name='gru'
)(x)

# Dropout
x = Dropout(0.3, name='dropout')(x)

# Output layer (dtype=float32 for numerical stability with mixed precision)
outputs = Dense(vocab_size, activation='softmax', dtype='float32', name='output')(x)

# Create model
model = Model(inputs=inputs, outputs=outputs, name='gru_keyboard')

# Compile
model.compile(
    optimizer=AdamW(
        learning_rate=1e-3,  # 0.001 (higher than Adam's default)
        weight_decay=1e-4    # Decoupled weight decay for better regularization
    ),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', Perplexity()]
)

model.summary()

params = model.count_params()
size_mb = (params * 4) / (1024 * 1024)

print("\n" + "="*60)
print("MODEL INFO")
print("="*60)
print(f"✓ Parameters: {params:,}")
print(f"✓ Size: {size_mb:.2f}MB (FP32), {size_mb/2:.2f}MB (FP16)")
print("✓ Architecture: Functional API")
print("✓ Optimizer: AdamW (lr=1e-3, weight_decay=1e-4)")
print("✓ Mixed Precision: Enabled")
print("="*60)

## 6. Train Model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

callbacks = [
    ModelCheckpoint(
        f'{DRIVE_DIR}/models/gru_keyboard/best_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        verbose=1
    )
]

print("="*60)
print("TRAINING (OPTIMIZED)")
print("="*60)
print(f"Epochs: {NUM_EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Mixed Precision: FP16")
print(f"Data Pipeline: tf.data (prefetched)")
print("="*60)

history = model.fit(
    train_dataset,
    epochs=NUM_EPOCHS,
    steps_per_epoch=train_steps,
    validation_data=val_dataset,
    validation_steps=val_steps,
    callbacks=callbacks,
    verbose=1
)

print("\n✓ Training complete!")
if TESTING_MODE:
    print("\n⚠️  This was TESTING mode")
    print("   Set TESTING_MODE = False for full training")

## 7. Visualize Training

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(history.history['loss'], label='Train')
ax1.plot(history.history['val_loss'], label='Val')
ax1.set_title('Loss')
ax1.legend()
ax1.grid(True)

ax2.plot(history.history['accuracy'], label='Train')
ax2.plot(history.history['val_accuracy'], label='Val')
ax2.set_title('Accuracy')
ax2.legend()
ax2.grid(True)

plt.show()

val_acc = history.history['val_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
print(f"\nFinal: Val Loss={val_loss:.4f}, Val Acc={val_acc*100:.2f}%")

## 8. Save Model

In [ ]:
import pickle

model.save(f'{DRIVE_DIR}/models/gru_keyboard/gru_model.keras')

with open(f'{DRIVE_DIR}/models/gru_keyboard/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

config = {'vocab_size': vocab_size, 'sequence_length': SEQUENCE_LENGTH}
with open(f'{DRIVE_DIR}/models/gru_keyboard/config.pkl', 'wb') as f:
    pickle.dump(config, f)

print("✓ Saved: gru_model.keras, tokenizer.pkl, config.pkl")

## 9. Test Predictions

In [ ]:
# ============================================================
# HYBRID PREDICTION SYSTEM (2026 UPGRADED)
# ============================================================
# - GRU model: next-word prediction
# - GRU reranking: context-aware typo/completion (NEW!)
# - Vocabulary filter: word completion
# - Edit distance: typo correction
# ============================================================

def edit_distance(s1, s2):
    """Calculate Levenshtein distance between two strings"""
    if len(s1) < len(s2):
        return edit_distance(s2, s1)
    if len(s2) == 0:
        return len(s1)
    
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]


# Build vocabulary list with word frequencies
vocab_list = sorted(tokenizer.word_index.items(), key=lambda x: x[1])[:VOCAB_SIZE_LIMIT]
vocab_words = [word for word, idx in vocab_list]
print(f"✓ Vocabulary loaded: {len(vocab_words):,} words")


def predict_next_word(context, top_k=5):
    """Use GRU model for next-word prediction"""
    seq = tokenizer.texts_to_sequences([context.lower()])[0]
    seq = seq[-SEQUENCE_LENGTH:]
    seq = pad_sequences([seq], maxlen=SEQUENCE_LENGTH, padding='pre')
    preds = model.predict(seq, verbose=0)[0]
    top_idx = np.argsort(preds)[-top_k:][::-1]
    return [(tokenizer.index_word.get(i, ''), preds[i]*100) for i in top_idx if i in tokenizer.index_word]


def complete_word(partial, top_k=5):
    """Use vocabulary filter for word completion"""
    partial = partial.lower()
    candidates = []
    
    for word in vocab_words:
        if word.startswith(partial) and word != partial:
            idx = tokenizer.word_index.get(word, 999999)
            score = 100 / (idx + 1)
            candidates.append((word, score))
    
    candidates.sort(key=lambda x: x[1], reverse=True)
    return candidates[:top_k]


def correct_typo(typo, top_k=5):
    """Use edit distance for typo correction"""
    typo = typo.lower()
    candidates = []
    
    for word in vocab_words[:10000]:
        if abs(len(word) - len(typo)) <= 2:
            dist = edit_distance(word, typo)
            if dist <= 2 and word != typo:
                idx = tokenizer.word_index.get(word, 999999)
                score = (100 / (dist + 1)) * (100 / (idx + 1))
                candidates.append((word, score, dist))
    
    candidates.sort(key=lambda x: x[1], reverse=True)
    return [(w, s, d) for w, s, d in candidates[:top_k]]


def rerank_with_gru(candidates, context, top_k=5):
    """
    2026 UPGRADE: Rerank candidates using GRU model probability.
    Makes completion/typo context-aware.
    """
    if not context or not candidates:
        return candidates
    
    # Get GRU predictions for context
    seq = tokenizer.texts_to_sequences([context.lower()])[0]
    seq = pad_sequences([seq], maxlen=SEQUENCE_LENGTH, padding='pre')
    preds = model.predict(seq, verbose=0)[0]
    
    scored_candidates = []
    for item in candidates:
        if len(item) == 3:
            word, score, task = item
        else:
            word, score = item[:2]
            task = 'unknown'
        
        # Get GRU probability for this word
        word_idx = tokenizer.word_index.get(word.lower(), 0)
        if word_idx > 0 and word_idx < len(preds):
            gru_prob = preds[word_idx] * 100
        else:
            gru_prob = 0
        
        # Combined score: 30% original + 70% GRU (context matters more)
        combined_score = score * 0.3 + gru_prob * 0.7
        scored_candidates.append((word, combined_score, task))
    
    scored_candidates.sort(key=lambda x: x[1], reverse=True)
    return scored_candidates[:top_k]


def predict_hybrid(input_text, top_k=5):
    """
    Hybrid prediction with GRU reranking (2026 upgrade):
    - "text " (ends with space) → GRU next-word prediction
    - "tex" (partial word) → Vocabulary completion + GRU rerank
    - "txet" (typo) → Edit distance + GRU rerank
    """
    results = []
    
    if input_text.endswith(' '):
        # NEXT-WORD PREDICTION (GRU model)
        task = 'next_word'
        preds = predict_next_word(input_text.strip(), top_k)
        results = [(word, prob, task) for word, prob in preds]
    
    else:
        words = input_text.split()
        if len(words) > 1:
            context = ' '.join(words[:-1])
            partial = words[-1]
        else:
            context = ""
            partial = input_text
        
        # Try COMPLETION first
        completions = complete_word(partial, top_k * 2)
        
        if completions:
            task = 'completion'
            results = [(word, score, task) for word, score in completions]
        else:
            # TYPO CORRECTION
            task = 'typo'
            corrections = correct_typo(partial, top_k * 2)
            results = [(word, score, task) for word, score, dist in corrections]
        
        # 2026 UPGRADE: RERANK with GRU if we have context
        if context and results:
            results = rerank_with_gru(results, context, top_k)
    
    return results[:top_k]


# ============================================================
# TEST HYBRID PREDICTIONS (2026 UPGRADED)
# ============================================================

test_cases = [
    # Next-word prediction (GRU)
    ("How are ", "Next-word (GRU)"),
    ("Thank ", "Next-word (GRU)"),
    ("I want to ", "Next-word (GRU)"),
    
    # Word completion (Vocab + GRU rerank)
    ("Hel", "Completion"),
    ("Tha", "Completion"),
    ("bea", "Completion"),
    
    # Typo correction (Edit dist + GRU rerank)
    ("thers", "Typo"),
    ("teh", "Typo"),
    ("helo", "Typo"),
    
    # Context + partial/typo (GRU RERANKING)
    ("How are yo", "Context + GRU Rerank"),
    ("How are thers", "Context + GRU Rerank"),
    ("I want to goe", "Context + GRU Rerank"),
]

print("\n" + "="*60)
print("HYBRID PREDICTION TESTS (2026 UPGRADED)")
print("="*60)

for input_text, test_type in test_cases:
    print(f"\n📝 Input: '{input_text}' ({test_type})")
    predictions = predict_hybrid(input_text, top_k=5)
    
    if not predictions:
        print("   (no predictions)")
        continue
    
    for i, (word, score, task) in enumerate(predictions, 1):
        if score > 50:
            emoji = "🟢"
        elif score > 10:
            emoji = "🟡"
        else:
            emoji = "🔴"
        
        print(f"  {i}. {word:15s} {emoji} {score:5.1f} [{task}]")

print("\n" + "="*60)
print("2026 Upgrades Applied:")
print("  ✓ GRU Reranking - Context-aware typo/completion")
print("  ✓ Perplexity Metric - Better quality measurement")
print("="*60)

## 10. Export to TFLite

In [ ]:
import tensorflow as tf
import numpy as np

print("Converting to TFLite with INT8 quantization (2026)...")
print("="*60)

# Representative dataset for INT8 calibration
def representative_dataset():
    """Generate representative data for INT8 calibration"""
    for _ in range(100):
        sample = np.random.randint(0, vocab_size, size=(1, SEQUENCE_LENGTH))
        yield [sample.astype(np.float32)]

# ============================================================
# 1. FP16 TFLite (standard)
# ============================================================
print("1. Creating FP16 TFLite...")

converter_fp16 = tf.lite.TFLiteConverter.from_keras_model(model)
converter_fp16.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter_fp16._experimental_lower_tensor_list_ops = False
converter_fp16.optimizations = [tf.lite.Optimize.DEFAULT]
converter_fp16.target_spec.supported_types = [tf.float16]

tflite_fp16 = converter_fp16.convert()
fp16_path = f'{DRIVE_DIR}/models/gru_keyboard/gru_model_fp16.tflite'
with open(fp16_path, 'wb') as f:
    f.write(tflite_fp16)

fp16_size = len(tflite_fp16) / (1024 * 1024)
print(f"   ✓ FP16 saved: {fp16_size:.2f}MB")

# ============================================================
# 2. INT8 TFLite (2026 recommended - smallest)
# ============================================================
print("2. Creating INT8 TFLite (2026 recommended)...")

try:
    converter_int8 = tf.lite.TFLiteConverter.from_keras_model(model)
    converter_int8.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    converter_int8._experimental_lower_tensor_list_ops = False
    converter_int8.optimizations = [tf.lite.Optimize.DEFAULT]
    converter_int8.representative_dataset = representative_dataset
    
    # Full INT8 quantization
    converter_int8.target_spec.supported_types = [tf.int8]
    converter_int8.inference_input_type = tf.int8
    converter_int8.inference_output_type = tf.int8
    
    tflite_int8 = converter_int8.convert()
    int8_path = f'{DRIVE_DIR}/models/gru_keyboard/gru_model_int8.tflite'
    with open(int8_path, 'wb') as f:
        f.write(tflite_int8)
    
    int8_size = len(tflite_int8) / (1024 * 1024)
    print(f"   ✓ INT8 saved: {int8_size:.2f}MB")
    
except Exception as e:
    print(f"   ⚠️ INT8 failed (GRU not fully supported): {str(e)[:50]}")
    print("   Using dynamic range quantization instead...")
    
    # Fallback: Dynamic range quantization
    converter_dyn = tf.lite.TFLiteConverter.from_keras_model(model)
    converter_dyn.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    converter_dyn._experimental_lower_tensor_list_ops = False
    converter_dyn.optimizations = [tf.lite.Optimize.DEFAULT]
    
    tflite_dyn = converter_dyn.convert()
    dyn_path = f'{DRIVE_DIR}/models/gru_keyboard/gru_model_optimized.tflite'
    with open(dyn_path, 'wb') as f:
        f.write(tflite_dyn)
    
    dyn_size = len(tflite_dyn) / (1024 * 1024)
    print(f"   ✓ Optimized saved: {dyn_size:.2f}MB")

# ============================================================
# Summary
# ============================================================
print("\n" + "="*60)

# ============================================================
# LATENCY BENCHMARK
# ============================================================
import time

print("Running latency benchmark...")

try:
    # Load the FP16 model for benchmarking
    interpreter = tf.lite.Interpreter(model_path=fp16_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    
    # Warm up
    for _ in range(5):
        test_input = np.random.randint(0, vocab_size, (1, SEQUENCE_LENGTH)).astype(np.float32)
        interpreter.set_tensor(input_details['index'], test_input)
        interpreter.invoke()
    
    # Benchmark 50 iterations
    latencies = []
    for _ in range(50):
        test_input = np.random.randint(0, vocab_size, (1, SEQUENCE_LENGTH)).astype(np.float32)
        interpreter.set_tensor(input_details['index'], test_input)
        
        start = time.time()
        interpreter.invoke()
        latencies.append((time.time() - start) * 1000)
    
    avg_latency = np.mean(latencies)
    min_latency = np.min(latencies)
    max_latency = np.max(latencies)
    
    print(f"\n✓ Latency benchmark (50 iterations):")
    print(f"   Average: {avg_latency:.2f}ms")
    print(f"   Min: {min_latency:.2f}ms")
    print(f"   Max: {max_latency:.2f}ms")
    
    if avg_latency < 10:
        print(f"   ✅ Good for mobile (<10ms target)")
    else:
        print(f"   ⚠️ May be slow on mobile (>10ms)")

except Exception as e:
    print(f"   ⚠️ Benchmark failed: {str(e)[:50]}")

print("TFLITE EXPORT SUMMARY")
print("="*60)
print(f"✓ FP16 model: {fp16_size:.2f}MB")
print(f"✓ Location: {DRIVE_DIR}/models/gru_keyboard/")
print("\n📱 For mobile deployment:")
print("   - Use FP16 for best accuracy")
print("   - Use INT8 for smallest size (if available)")
print("   - Note: GRU requires SELECT_TF_OPS")
print("\n🍎 For iOS: Run next cell to export CoreML (.mlpackage)")
print("="*60)
print("\n🎉 Training complete! Download from Google Drive.")

## 11. Export to CoreML (iOS Native)

Export the model to CoreML format for optimal iOS performance:
- ✅ Native Neural Engine support
- ✅ No TFLite SDK required
- ✅ Fastest inference on iOS

In [ ]:
# ============================================================
# COREML EXPORT (iOS Native)
# ============================================================

print("Installing coremltools (compatible version)...")
!pip install -q coremltools==7.2 protobuf==3.20.3

import warnings
warnings.filterwarnings('ignore')  # Suppress version warnings

import coremltools as ct
import numpy as np
import tensorflow as tf

print("\n" + "="*60)
print("COREML EXPORT FOR iOS")
print("="*60)

# Configuration (must match training config)
seq_length = 10    # SEQUENCE_LENGTH from config cell
vocab_size = 25000 # VOCAB_SIZE_LIMIT from config cell

# Load model from saved .keras file
keras_model_path = f'{DRIVE_DIR}/models/gru_keyboard/gru_model.keras'
print(f"Loading model from: {keras_model_path}")

try:
    keras_model = tf.keras.models.load_model(keras_model_path)
    print(f"✓ Model loaded successfully")
    print(f"  Input shape: {keras_model.input_shape}")
    print(f"  Output shape: {keras_model.output_shape}")
except Exception as e:
    print(f"✗ Failed to load model: {e}")
    print("  Make sure cell '8. Save Model' was executed first")
    raise e

try:
    print("\nConverting model to CoreML...")
    
    # Convert Keras model to CoreML
    mlmodel = ct.convert(
        keras_model,
        inputs=[ct.TensorType(
            shape=(1, seq_length),
            name="input",
            dtype=np.float32
        )],
        outputs=[ct.TensorType(name="output")],
        convert_to="mlprogram",
        compute_precision=ct.precision.FLOAT16,
        minimum_deployment_target=ct.target.iOS15
    )
    
    # Add metadata
    mlmodel.author = "Keyboard-Suggestions-ML"
    mlmodel.short_description = "GRU Keyboard Prediction Model - Next word, completion, typo correction"
    mlmodel.version = "1.0.0"
    
    # Save as .mlpackage
    coreml_path = f'{DRIVE_DIR}/models/gru_keyboard/gru_keyboard_ios.mlpackage'
    mlmodel.save(coreml_path)
    
    # Get size
    import os
    def get_dir_size(path):
        total = 0
        for dirpath, dirnames, filenames in os.walk(path):
            for f in filenames:
                fp = os.path.join(dirpath, f)
                total += os.path.getsize(fp)
        return total
    
    coreml_size = get_dir_size(coreml_path) / (1024 * 1024)
    
    print(f"\n✅ CoreML Export Success!")
    print(f"\n" + "="*60)
    print("COREML MODEL INFO")
    print("="*60)
    print(f"✓ Format: .mlpackage (ML Program)")
    print(f"✓ Size: {coreml_size:.2f}MB")
    print(f"✓ Precision: Float16")
    print(f"✓ Input: (1, {seq_length}) float32")
    print(f"✓ Output: (1, {vocab_size}) float32")
    print(f"✓ Target: iOS 15+")
    print(f"✓ Path: {coreml_path}")
    print("\n📱 iOS Deployment:")
    print("   1. Download gru_keyboard_ios.mlpackage from Drive")
    print("   2. Drag .mlpackage to Xcode project")
    print("   3. Uses Neural Engine automatically")
    print("   4. No TensorFlow SDK required!")
    print("="*60)
    
except Exception as e:
    print(f"\n⚠️ CoreML conversion failed: {str(e)}")
    print("\nPossible causes:")
    print("  1. GRU layer conversion issue")
    print("  2. TensorFlow/coremltools version mismatch")
    print("\nSolution: Use TFLite on iOS instead.")
    print("  - Download gru_model_fp16.tflite")
    print("  - Add TensorFlowLiteSwift + SelectTfOps pods")